In [1]:
import os
from tempfile import NamedTemporaryFile

os.chdir("/local/")

from contact_utils import (
    FileType,
    ResiduesSelect,
    get_file,
    split_structure_to_hermetic_chains,
    structure_cif_filter,
)
from find_contact import (
    InContactResult,
    find_contact_hybrid_ligand_protein,
    find_contact_ion,
    get_dot_bracket_and_filter,
    get_dssp_and_filter,
)
from pymol import cmd


pdb_id = "1PJG"
model = 1
chains = ["A"]

result = InContactResult()
find_contact_hybrid_ligand_protein(pdb_id, model, chains, result)
find_contact_ion(pdb_id, model, chains, result)

structure_file_path_cif, _ = get_file(
    pdb_id, model, FileType.CIF, save_ions=True, ligands=True
)
rna_file = NamedTemporaryFile(suffix="_RNA.cif", delete=False)
dna_file = NamedTemporaryFile(suffix="_DNA.cif", delete=False)
protein_file = NamedTemporaryFile(suffix="_PROTEIN.cif", delete=False)
ion_file = NamedTemporaryFile(suffix="_ION.cif", delete=False)
split_structure_to_hermetic_chains(
    structure_file_path_cif,
    [rna_file.name, dna_file.name, protein_file.name, ion_file.name],
    model,
    result.in_contact_desc
)

result.dot_bracket_file_path = get_dot_bracket_and_filter(
    rna_file.name, result.in_contact
)

cmd.delete("all")
cmd.load(rna_file.name, f"{pdb_id.upper()}_RNA", quiet=1)
cmd.save(rna_file.name.split(".")[0] + ".fasta", quiet=1)
result.rna_file_cif = rna_file.name
result.rna_file_fasta = rna_file.name.split(".")[0] + ".fasta"

try:
    structure_cif_filter(dna_file.name, ResiduesSelect(result.in_contact))
    cmd.delete("all")
    cmd.load(dna_file.name, f"{pdb_id.upper()}_DNA", quiet=1)
    cmd.save(dna_file.name.split(".")[0] + ".fasta", quiet=1)
    result.dna_file_cif = dna_file.name
    result.dna_file_fasta = dna_file.name.split(".")[0] + ".fasta"
except ValueError:
    os.remove(dna_file.name)
try:
    get_dssp_and_filter(protein_file.name, result.in_contact)
    cmd.delete("all")
    cmd.load(protein_file.name, f"{pdb_id.upper()}_PROTEIN", quiet=1)
    cmd.save(protein_file.name.split(".")[0] + ".fasta", quiet=1)
    result.protein_file_cif = protein_file.name
    result.protein_file_fasta = protein_file.name.split(".")[0] + ".fasta"
except ValueError:
    os.remove(protein_file.name)

try:
    structure_cif_filter(ion_file.name, ResiduesSelect(result.in_contact))
    cmd.delete("all")
    cmd.load(ion_file.name, f"{pdb_id.upper()}_ION", quiet=1)
    cmd.save(ion_file.name.split(".")[0] + ".fasta", quiet=1)
    result.ion_motif_cif = ion_file.name
    result.ion_motif_fasta = ion_file.name.split(".")[0] + ".fasta"
except ValueError:
    os.remove(ion_file.name)

structure_cif_filter(structure_file_path_cif, ResiduesSelect(result.in_contact))
cmd.delete("all")
cmd.load(structure_file_path_cif, f"{pdb_id.upper()}_FULL_MOTIF", quiet=1)
cmd.save(structure_file_path_cif.split(".")[0] + ".fasta", quiet=1)
result.full_motif_cif = structure_file_path_cif
result.full_motif_fasta = structure_file_path_cif.split(".")[0] + ".fasta"

In [3]:
structure_cif_filter(structure_file_path_cif, ResiduesSelect(result.in_contact))

A.33 True {'B.18', 'A.4', 'B.11', 'B.16', 'B.17', 'B.13', 'A.7', 'A.1', 'A.33', 'A.10', 'A.6', 'A.8', 'B.15', 'A.5', 'A.9', 'A.3', 'B.31', 'B.20', 'B.34', 'B.14', 'B.12', 'A.2', 'B.19'}


In [6]:
result.in_contact_desc

{'A.1': [('B.20', 'DG', <MoleculeType.DNA: 0>)],
 'A.2': [('B.19', 'DT', <MoleculeType.DNA: 0>)],
 'A.3': [('B.18', 'DT', <MoleculeType.DNA: 0>)],
 'A.4': [('B.17', 'DT', <MoleculeType.DNA: 0>)],
 'A.5': [('B.16', 'DC', <MoleculeType.DNA: 0>)],
 'A.6': [('B.15', 'DT', <MoleculeType.DNA: 0>)],
 'A.7': [('B.14', 'DT', <MoleculeType.DNA: 0>)],
 'A.8': [('B.13', 'DT', <MoleculeType.DNA: 0>)],
 'A.9': [('B.12', 'DT', <MoleculeType.DNA: 0>),
  ['A.33', 'CA', <MoleculeType.ION: 3>]],
 'A.10': [('B.11', 'DC', <MoleculeType.DNA: 0>)]}

In [5]:
!cat {rna_file.name}

data_str
#
loop_
_atom_site.group_PDB
_atom_site.id
_atom_site.type_symbol
_atom_site.label_atom_id
_atom_site.label_alt_id
_atom_site.label_comp_id
_atom_site.label_asym_id
_atom_site.label_entity_id
_atom_site.label_seq_id
_atom_site.pdbx_PDB_ins_code
_atom_site.Cartn_x
_atom_site.Cartn_y
_atom_site.Cartn_z
_atom_site.occupancy
_atom_site.B_iso_or_equiv
_atom_site.auth_seq_id
_atom_site.auth_asym_id
_atom_site.pdbx_PDB_model_num
ATOM 1   O 'O5''  . C A ? 1  ? 15.621 24.330 29.094 1.0  14.55 1  A 1 
ATOM 2   C 'C5''  . C A ? 1  ? 14.673 25.355 29.347 1.0  13.57 1  A 1 
ATOM 3   C 'C4''  . C A ? 1  ? 13.307 24.862 29.746 1.0  12.5  1  A 1 
ATOM 4   O 'O4''  . C A ? 1  ? 13.236 24.374 31.104 1.0  12.05 1  A 1 
ATOM 5   C 'C3''  . C A ? 1  ? 12.259 25.970 29.706 1.0  12.35 1  A 1 
ATOM 6   O 'O3''  . C A ? 1  ? 11.784 26.105 28.391 1.0  13.95 1  A 1 
ATOM 7   C 'C2''  . C A ? 1  ? 11.150 25.472 30.603 1.0  12.18 1  A 1 
ATOM 8   O 'O2''  . C A ? 1  ? 10.428 24.389 30.013 1.0  12.94 1  A 

In [2]:
import pandas as pd

from hbplus_process_cif import MoleculeType





In [ ]:
from hbplus_process_cif import MoleculeType




In [4]:
in_contact_desc

{}